In [ ]:
us vs rest of world.. about outsiders perspective on the us

## Setting up google drive connection to access twitter api information (saved as separate file)

In [ ]:
import tweepy 

In [ ]:

!pip install -U -q PyDrive tweepy
import os 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
def download_file(file_id, file_name):
    '''
    file_id: Make sure that you can access this file from your account or else, this will not run
    file_name: Name of the file by which you want to save
    '''
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(file_name)
    print('Downloaded file with ID {} and name {}'.format(file_id, file_name))

In [ ]:
if not os.path.isfile('secrets.txt'):
    download_file('1NN7yme39CJoQmsIrcXyLs0mLg131cF2A', 'secrets.txt')

Downloaded file with ID 1NN7yme39CJoQmsIrcXyLs0mLg131cF2A and name secrets.txt


In [ ]:
with open('secrets.txt', 'r') as file:
    data = file.read().split('\n')

import tweepy 
import pandas as pd

consumer_key = data[0]
consumer_secret = data[1]
access_key = data[2]
access_secret = data[3]

In [ ]:
#list of states

states = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'DC': 'District of Columbia',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming'
}

#look for USA in location or these 

## following function grabs tweets based off a search value:
 my understanding is that it will expand the breadth of our search to include users that are talking about the candidates without using something as formal as a hashtag

 search things: biden2020, bidenharris2020, trump2020

 we have to keep in mind the original investigation is on swing states political affiliation and if trump getting covid is impacting that

 bc since we are using data around the political climate I feel like it’s important to scrape all the hashtags at the same time even though we may acquire less data per tag as it will be more consistent with respect of time Idk

In [ ]:
def get_query_tweets(query, num=0):
    num = 3000 if num > 3000 else num
    max_num_per_call = 100

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []    
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    curr_count = max_num_per_call if num > max_num_per_call else num
    num -= curr_count

    new_tweets = api.search(q=query, count=curr_count)
    
    #save most recent tweets
    alltweets.extend(new_tweets)

    #sort all tweets to make sure oldest is grabbing the oldest one 
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #print(f"{len(alltweets)} tweets downloaded so far")

    #keep grabbing tweets until there are no tweets left to grab
    while num > 0:
        #print(f"Getting tweets before {oldest}")
        
        curr_count = max_num_per_call if num > max_num_per_call else num

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.search(q=query, count=curr_count, max_id=oldest)
        num -= curr_count
        
        #save most recent tweets
        alltweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        
        #print(f"{len(alltweets)} tweets downloaded so far")
    
    #transform the tweepy tweets into a 2D array that will populate the csv    
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8"), tweet.user.location] for tweet in alltweets]
    df = pd.DataFrame(outtweets, columns=["id", "created_at", "text", "location"])
    #df.to_csv(f"query_{query}.csv", index=False)
    #print(df.head())
    return df

creating empty dataframes to store our tweets. only do this once initially otherwise import saved csv files

In [ ]:

biden2020 = pd.DataFrame(columns=["id", "created_at", "text", "location"])
bidenharris2020 = pd.DataFrame(columns=["id", "created_at", "text", "location"])
trump2020 = pd.DataFrame(columns=["id", "created_at", "text", "location"])
kag2020 = pd.DataFrame(columns=["id", "created_at", "text", "location"])


In [ ]:
trump = pd.DataFrame(columns=["id", "created_at", "text", "location"])
biden = pd.DataFrame(columns=["id", "created_at", "text", "location"])

In [ ]:
new = pd.DataFrame(columns=["id", "created_at", "text", "location"])

In [ ]:
row = {'id':1,'created_at':2,'text':3,'location':4}
new = new.append(row,ignore_index=True)

In [ ]:
new

,id,created_at,text,location
0,1,2,3,4
1,1,2,3,4
2,1,2,3,4


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving biden.csv to biden.csv


In [ ]:
import io
import pandas as pd 


# Dataset is now stored in a Pandas Dataframe

In [ ]:
trump = pd.read_csv(io.BytesIO(uploaded['trump.csv']))

In [ ]:
biden = pd.read_csv(io.BytesIO(uploaded['biden.csv']))

In [ ]:
bidenharris2020 = pd.read_csv(io.BytesIO(uploaded['bidenharris2020.csv']))

In [ ]:
bidenharris2020.head()

,id,created_at,text,location
0,1313518495603228672,2020-10-06 16:36:28,b'RT @ASky1113: @realDonaldTrump #Biden2020 be...,"British Columbia, Canada"
1,1313518475915087872,2020-10-06 16:36:23,b'#Biden2020 https://t.co/FavNLNIfEt',Tucson
2,1313518463642742784,2020-10-06 16:36:20,"b'RT @gayleasher1: Tired of the West burning, ...",NaN
3,1313518442205646849,2020-10-06 16:36:15,b'RT @krupali: My pup is in the @JoeBiden VIDE...,NaN
4,1313518433779118080,2020-10-06 16:36:13,b'I got my vote-by-mail ballot &amp; I will vo...,"South Central, Los Angeles"


In [ ]:
index = 0
while index <= 130: #grabs 15 tweets at a time
  #populating biden2020 
  biden_query = get_query_tweets('biden') #querying tweets
  biden = biden.append(biden_query, ignore_index=True) #appending to current dataframe 

  #populating trump2020
  trump_query = get_query_tweets('trump') #querying tweets
  trump = trump.append(trump_query, ignore_index=True) 


  index +=1 #incrementing index 

RateLimitError: ignored

In [ ]:
 index = 0
while index <= 130: #grabs 15 tweets at a time
  #populating biden2020 
  biden = get_query_tweets('biden2020') #querying tweets
  biden2020 = biden2020.append(biden, ignore_index=True) #appending to current dataframe 

  #populating bidenharris2020
  bidenharris = get_query_tweets('bidenharris2020') #querying tweets
  bidenharris2020 = bidenharris2020.append(bidenharris, ignore_index=True) 

  #populating trump2020
  trump = get_query_tweets('trump2020') #querying tweets
  trump2020 = trump2020.append(trump, ignore_index=True) 

  #populating MAGA2020
  kag = get_query_tweets('KAG') #querying tweets
  kag2020 = kag2020.append(kag, ignore_index=True) 

  index +=1 #incrementing index 

In [ ]:
print(len(biden2020))
print(len(bidenharris2020))
print(len(trump2020))
print(len(kag2020))

22
57
31
24


In [ ]:
print(len(biden))
print(len(trump))

3819
4321


In [ ]:
biden.head()

,id,created_at,text,location
0,1313832962065137664,2020-10-07 13:26:02,b'RT @GeorgeMonbiot: We can\xe2\x80\x99t expec...,"Philadelphia, PA"
1,1313832961263898629,2020-10-07 13:26:02,b'RT @BreitbartNews: Russian President Vladimi...,
2,1313832961238859781,2020-10-07 13:26:02,b'RT @matthewjdowd: This speech Joe Biden just...,
3,1313832961062694926,2020-10-07 13:26:02,b'It would be very on brand of 2020 for Biden ...,
4,1313832960882348035,2020-10-07 13:26:02,b'RT @my3monkees: So we now know Crooked Hilla...,Somewhere in pleasantville!!!!


In [ ]:
biden.to_csv('biden.csv',index=False)
trump.to_csv('trump.csv',index=False)

In [ ]:
biden2020.to_csv('biden2020.csv',index=False)
bidenharris2020.to_csv('bidenharris2020.csv',index=False)
trump2020.to_csv('trump2020.csv',index=False)
kag2020.to_csv('kag2020.csv',index=False)

In [ ]:
trump2020.drop_duplicates(inplace=True)
biden2020.drop_duplicates(inplace=True)
bidenharris2020.drop_duplicates(inplace=True)
kag2020.drop_duplicates(inplace=True)

In [ ]:
trump.drop_duplicates(inplace=True)
biden.drop_duplicates(inplace=True)


In [ ]:
biden['location'].value_counts()

                            178
United States                 8
Texas, USA                    6
Florida, USA                  5
USA                           4
                           ... 
Caruaru, Brasil               1
She/Her Minor 🇺🇸              1
New York Cit-ay!              1
Northern California, USA      1
Michigan_LA_Earth             1
Name: location, Length: 163, dtype: int64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
kag2020['text'][5]

NameError: ignored

In [ ]:
biden['text'][1000]

"b'@E_PIuribusUnum @Mooney4me @funder No, she was not the only woman to be chosen. Most Likely it would be another wom\\xe2\\x80\\xa6 https://t.co/Dxo2ESL1ty'"

In [ ]:
dups = biden[biden.duplicated() == True]

In [ ]:
dups[dups['id']==	'1313881557585793025']['text']

NameError: ignored